Defining properties of the 2DOF systems of two springs in series

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider


m1 = 1.
m2 = 1.
k1 = 1.e3 # stiff spring
k2 = 1.

M = np.array([[m1, 0],
              [0, m2]])

K = np.array([[k1+k2, -k2],
              [-k2, k2]])

C = np.zeros_like(M)

Calculating initial acceleration

In [ ]:
u0 = np.array([1, 20])
v0 = np.array([0, 0])

tmax = 50
t = np.linspace(0, tmax, 200) # Delta t = 25/100
dt = t[1] - t[0]

u = np.zeros((2, t.shape[0]))
v = np.zeros((2, t.shape[0]))
a = np.zeros((2, t.shape[0]))

u[:, 0] = u0
v[:, 0] = v0
a[:, 0] = np.linalg.solve(M, - C @ v0 - K @ u0)

Direct single-step method

In [ ]:
def effect_of_spectral_radius(rho_infty=0.5):
#if True:
    #rho_infty = 0.5
    alpham = (2*rho_infty - 1)/(rho_infty + 1)
    alphaf = rho_infty/(rho_infty + 1)
    beta = 1/4*(1 + alphaf - alpham)**2
    gamma = 1/2 + (alphaf - alpham)
    
    
    for i in range(0, t.shape[0]-1):
        B = (1 - alpham)*M + gamma*dt*(1 - alphaf)*C + beta*dt**2*(1 - alphaf)*K
        
        # predictor
        uP = u[:, i] + dt*v[:, i] + dt**2/2*(1 - 2*beta)*a[:, i]
        vP = v[:, i] + (1 - gamma)*dt*a[:, i]
        
        b = - (1 - alphaf)*C@vP - (1 - alphaf)*K@uP - alphaf*K@u[:, i] - alphaf*C@v[:, i] - alpham*M@a[:, i]
        
        a[:, i+1] = np.linalg.solve(B, b)
        
        # corrector
        u[:, i+1] = uP + beta*dt**2*a[:, i+1]
        v[:, i+1] = vP + gamma*dt*a[:, i+1]
    
    x1 = u[0, :]
    x2 = u[1, :]
    fig, axes = plt.subplots(ncols=2, figsize=(10, 5))
    axes[0].plot(t, x1)
    axes[0].set_xlabel('time')
    axes[0].set_ylabel('$x_1$', rotation='horizontal')
    axes[0].set_ylim(-1, 1)
    
    axes[1].plot(t, x2)
    axes[1].set_xlabel('time')
    axes[1].set_ylabel('$x_2$', rotation='horizontal')
    axes[1].set_ylim(-20, 20)
    #plt.show()


interact(effect_of_spectral_radius, rho_infty=FloatSlider(value=0.5, min=0, max=1, step=0.1, 
                                                          description=r'$\rho_\infty$'));